In [1]:
import telegram
from telegram.ext import Updater
from telegram.ext import CommandHandler
import logging

%run token.py

updater = Updater(token=token_id, use_context=True)
dispatcher = updater.dispatcher

In [2]:
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',level=logging.INFO)

In [3]:
start_text = '''
    Olá! \n
    Sou um bot de notícias e informações sobre a Covid-19!\n
    O serviço de jornalismo do Instituto Oswaldo Cruz fez a triagem do conteúdo aqui.
    Sou um bot repassador não oficial! Recebo deles e repasso para vocês.\n
    Não conhece o Instituto Oswaldo Cruz? \n
    Entre em www.ioc.fiocruz.br para conhecer! \n
    (Bot não-oficial administrado por Tiago Lubiana)\n

    (em desenvolvimento)\n"
'''

In [4]:
def start(update, context):
        context.bot.send_message(chat_id=update.effective_chat.id,
                                 text=start_text)
        
start_handler = CommandHandler('start', start)
dispatcher.add_handler(start_handler)

In [5]:
with open('14_05_2020_p1.txt', 'r') as file_1:
        comunicado_p1 = file_1.read()
        comunicado_parseado_p1 = comunicado_p1.split("--")
        
        
with open('14_05_2020_p2.txt', 'r') as file_2:
        comunicado_p2 = file_2.read()
        comunicado_parseado_p2 = comunicado_p2.split("--")
        

In [6]:
def get_header(section):
    raw_header = section.split("\n\n")[0]
    header = raw_header.replace("\n", "")
    header = header.replace("\U0001f97c ", "")
    header = header.replace("\U0001f9ea", "")
    return(header)

def get_section_content(section):
    raw_content = section[1:]
    return(raw_content)

def get_header_section_dict(comunicado_parseado):
    comunicado_headers = [get_header(section) for section in comunicado_parseado]
    comunicado_sections = [get_section_content(section) for section in comunicado_parseado]
    comunicado_header_to_section = dict(zip(comunicado_headers, comunicado_sections ))
    return(comunicado_header_to_section)

In [7]:
possible_section_headers= ['🔬 Pesquisa científica', ' Diagnóstico', '📊 Epidemiologia', '💊 Tratamento', 
 '🏡 Distanciamento social', '🗂️ Política', 'Saúde Pública', '🌐 Internacional', 
 '🔍 Recortes da pandemia',  '✅ Curtas', 
 'Produzido pelo Serviço de Jornalismo e Comunicação do Instituto Oswaldo Cruz (IOC/Fiocruz)']

In [10]:
d = get_header_section_dict(comunicado_parseado_p1)
d1  = get_header_section_dict(comunicado_parseado_p2)

# def concatenate_dicts(d,d1):
data_da_atualização = "15 de maio de 2020"
comunicado_completo_de_hoje = {}


for section_header in possible_section_headers:
    try:
        section_part_1 = (" - Parte 1 - " + data_da_atualização + "\n\n" + d[section_header]) 
    except: 
        print(" Parte 1 ausente ou sem " + section_header)
        
    try:
        section_part_2 = (" - Parte 2 - " + data_da_atualização + "\n\n" + d1[section_header])
    except: 
        print(" Parte 2 ausente ou sem " + section_header)
        
    try:
        comunicado_completo_de_hoje[section_header] = section_part_1 + "\n\n" + section_part_2     
    except:
        
        try:
            comunicado_completo_de_hoje[section_header] = section_part_1
        except:
            
            try:
            comunicado_completo_de_hoje[section_header] = section_part_2
                 
            except:
                comunicado_completo_de_hoje[section_header] = "Seção não está presente no comunicado de hoje."
            

 Parte 1 ausente ou sem 💊 Tratamento


In [7]:
comunicado_headers = [get_header(section) for section in comunicado_parseado]
comunicado_sections = [get_section_content(section) for section in comunicado_parseado]
comunicado_header_to_section = dict(zip(comunicado_headers, comunicado_sections ))

In [8]:
comunicado_headers

['Destaques - 14/05/2020',
 '📊 Epidemiologia',
 '🔬 Pesquisa científica',
 ' Diagnóstico',
 '🏡 Distanciamento social',
 '🗂️ Política',
 'Saúde Pública',
 '🌐 Internacional',
 '🔍 Recortes da pandemia',
 '✅ Curtas',
 'Produzido pelo Serviço de Jornalismo e Comunicação do Instituto Oswaldo Cruz (IOC/Fiocruz)']

In [9]:
updater.start_polling()

In [10]:
# Repetição de código pois não conheço tão bem o módulo telegram
# O código grita por refatoração. 


def epid(update, context):
    text = comunicado_header_to_section["📊 Epidemiologia"]
    context.bot.send_message(chat_id=update.effective_chat.id,
                             text=text)
    
handler_for_this_call = CommandHandler('epid', epid)
dispatcher.add_handler(handler_for_this_call)

In [11]:
def ajuda(update, context):
    text = '''
    O robô possui atualmente funções para os seguintes temas
    - 📊 Epidemiologia (/epid)
    - 🔬 Pesquisa científica (/pesquisa)
    - Diagnóstico (/diag)
    - 🏡 Distanciamento social (/distanciamento)
    - 🗂️ Política (/politica)
    - Saúde Pública (/saude)
    - 🌐 Internacional (/internacional)
    - 🔍 Recortes da pandemia (/recortes)
    - ✅ Curtas (/curtas)  
    '''
    context.bot.send_message(chat_id=update.effective_chat.id,
                             text=text)
    
ajuda_handler = CommandHandler('ajuda', ajuda)
dispatcher.add_handler(ajuda_handler)

In [12]:
def diag(update, context):
    text = comunicado_header_to_section[" Diagnóstico"]
    context.bot.send_message(chat_id=update.effective_chat.id,
                             text=text)
    
handler_for_this_call = CommandHandler('diag', diag)
dispatcher.add_handler(handler_for_this_call)

In [13]:
def politica(update, context):
    text = comunicado_header_to_section["🗂️ Política"]
    context.bot.send_message(chat_id=update.effective_chat.id,
                             text=text)
    
handler_for_this_call = CommandHandler('politica', politica)
dispatcher.add_handler(handler_for_this_call)

In [14]:
def internacional(update, context):
    text = comunicado_header_to_section["🌐 Internacional"]
    context.bot.send_message(chat_id=update.effective_chat.id,
                             text=text)
    
handler_for_this_call = CommandHandler('internacional', internacional)
dispatcher.add_handler(handler_for_this_call)

In [15]:
def recortes(update, context):
    text = comunicado_header_to_section['🔍 Recortes da pandemia']
    context.bot.send_message(chat_id=update.effective_chat.id,
                             text=text)
    
handler_for_this_call = CommandHandler('recortes', recortes)
dispatcher.add_handler(handler_for_this_call)

In [16]:
def curtas(update, context):
    text = comunicado_header_to_section['✅ Curtas']
    context.bot.send_message(chat_id=update.effective_chat.id,
                             text=text)
    
handler_for_this_call = CommandHandler('curtas', curtas)
dispatcher.add_handler(handler_for_this_call)

In [17]:
def distanciamento(update, context):
    text = comunicado_header_to_section['🏡 Distanciamento social']
    context.bot.send_message(chat_id=update.effective_chat.id,
                             text=text)
    
handler_for_this_call = CommandHandler('distanciamento', distanciamento)
dispatcher.add_handler(handler_for_this_call)

In [18]:
def pesquisa(update, context):
    text = comunicado_header_to_section['🔬 Pesquisa científica']
    context.bot.send_message(chat_id=update.effective_chat.id,
                             text=text)
    
handler_for_this_call = CommandHandler('pesquisa', pesquisa)
dispatcher.add_handler(handler_for_this_call)

In [19]:
def saude(update, context):
    text = comunicado_header_to_section['Saúde Pública']
    context.bot.send_message(chat_id=update.effective_chat.id,
                             text=text)
    
handler_for_this_call = CommandHandler('saude', saude)
dispatcher.add_handler(handler_for_this_call)